In [13]:
import os

print(os.getcwd())

C:\Users\gopij


In [24]:
print(os.listdir('.'))
#help(os.listdir)
'My Documents' in os.listdir('.')

['.android', '.bash_history', '.cache', '.config', '.eclipse', '.git', '.gitconfig', '.gradle', '.ipynb_checkpoints', '.ipython', '.jupyter', '.lemminx', '.lesshst', '.m2', '.rediscli_history', '.ssh', '.sts4', '.viminfo', '.vscode', 'AppData', 'Application Data', 'Contacts', 'Cookies', 'demo', 'Documents', 'Downloads', 'Favorites', 'git', 'IBMDS.ipynb', 'Links', 'Local Settings', 'Music', 'My Documents', 'NetHood', 'netstat', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{2ad838bc-efea-11ee-a54d-000d3a94eaa1}.TM.blf', 'NTUSER.DAT{2ad838bc-efea-11ee-a54d-000d3a94eaa1}.TMContainer00000000000000000001.regtrans-ms', 'NTUSER.DAT{2ad838bc-efea-11ee-a54d-000d3a94eaa1}.TMContainer00000000000000000002.regtrans-ms', 'ntuser.ini', 'OneDrive', 'Postman', 'PrintHood', 'PycharmProjects', 'Recent', 'Saved Games', 'Searches', 'SendTo', 'Start Menu', 'Templates', 'Videos']


True

In [25]:
os.makedirs('./data')
'data' in os.listdir('.')

True

In [27]:
url1 = 'https://gist.githubusercontent.com/aakashns/257f6e6c8719c17d0e498ea287d1a386/raw/7def9ef4234ddf0bc82f855ad67dac8b971852ef/loans1.txt'
url2 = 'https://gist.githubusercontent.com/aakashns/257f6e6c8719c17d0e498ea287d1a386/raw/7def9ef4234ddf0bc82f855ad67dac8b971852ef/loans2.txt'
url3 = 'https://gist.githubusercontent.com/aakashns/257f6e6c8719c17d0e498ea287d1a386/raw/7def9ef4234ddf0bc82f855ad67dac8b971852ef/loans3.txt'

In [31]:
from urllib.request import urlretrieve
import pandas as pd

urlretrieve(url1,'./data/loans1.txt')
urlretrieve(url2,'./data/loans2.txt')
urlretrieve(url3,'./data/loans3.txt')
os.listdir('./data')

['loans1.txt', 'loans2.txt', 'loans3.txt']

In [32]:
with open('./data/loans1.txt') as file1:
    file1_content=file1.read()
    print(file1_content)

amount,duration,rate,down_payment
100000,36,0.08,20000
200000,12,0.1,
628400,120,0.12,100000
4637400,240,0.06,
42900,90,0.07,8900
916000,16,0.13,
45230,48,0.08,4300
991360,99,0.08,
423000,27,0.09,47200


In [53]:
with open('./data/loans3.txt','r') as file3:
    file3_content = file3.readlines()

print(file3_content)

def readHeader(header_lines):
    return header_lines.strip().split(',')

def parse_values(data_lines):
    values=[]
    for item in data_lines.strip().split(','):
        if item == '':
            values.append(0.0)
        else:
            try:
                values.append(float(item))
            except ValueError:
                values.append(item)
    return values

['amount,duration,rate,down_payment\n', '45230,48,0.07,4300\n', '883000,16,0.14,\n', '100000,12,0.1,\n', '728400,120,0.12,100000\n', '3637400,240,0.06,\n', '82900,90,0.07,8900\n', '316000,16,0.13,\n', '15230,48,0.08,4300\n', '991360,99,0.08,\n', '323000,27,0.09,4720010000,36,0.08,20000\n', '528400,120,0.11,100000\n', '8633400,240,0.06,\n', '12900,90,0.08,8900']


In [56]:

header=readHeader(file3_content[0])
print(header)

['amount', 'duration', 'rate', 'down_payment']


In [58]:
file3_line2=parse_values(file3_content[1])
print(file3_line2)

file3_line3=parse_values(file3_content[2])
print(file3_line3)

[45230.0, 48.0, 0.07, 4300.0]
[883000.0, 16.0, 0.14, 0.0]


In [68]:
def create_item_dict(values,header):
    result={}
    for val,head in zip(values,header):
        result[head]=val
    return result

In [64]:
def read_csv(path):
    result=[]
    with open(path,'r') as f:
       lines= f.readlines()

       headers = readHeader(lines[0])

       for data_line in lines[1:]:
           values = parse_values(data_line)
           item_dict=create_item_dict(values,headers)

           result.append(item_dict)
    return result

In [71]:
loans2=read_csv('./data/loans3.txt')


In [70]:
import math

def loan_emi(amount,duration,rate,down_pay=0):
    """Calculates the EMI for a loan"""
    loan_amount = amount - down_pay
    try:
        emi = loan_amount * rate * ((1+rate)**duration) / (((1+rate)**duration)-1)
    except ZeroDivisionError:
        emi = loan_amount / duration
    emi = math.ceil(emi)
    return emi


In [73]:
for loan in loans2:
    loan['emi']=loan_emi(loan["amount"],loan["duration"],loan['rate']/12,loan['down_payment'])

print(loans2[0]['emi'])

981


In [75]:
def compute_emi(loans):
    for loan in loans:
         loan['emi']=loan_emi(loan["amount"],loan["duration"],loan['rate']/12,loan['down_payment'])

In [77]:
with open('./data/emis2.txt','w') as f:
    for loan in loans2:
        f.write('{},{},{},{},{}\n'.format(loan['amount'], 
            loan['duration'], 
            loan['rate'], 
            loan['down_payment'], 
            loan['emi']))

In [78]:
os.listdir('data')

['emis2.txt', 'loans1.txt', 'loans2.txt', 'loans3.txt']

In [82]:
def write_csv(items,path):
    with open(path,'w') as f:
        if len(items)==0:
            return
        
        headers = list(items[0].keys())
        f.write(','.join(headers)+'\n')

        for item in items:
            values=[]
            for head in headers:
                values.append(str(item.get(head,"")))
            f.write(','.join(values)+'\n')

In [86]:
for i in range(1,4):
    loans = read_csv('./data/loans{}.txt'.format(i))
    compute_emi(loans)
    write_csv(loans,'./data/emis{}.csv'.format(i))

In [85]:
os.listdir('data')
os.getcwd()

'C:\\Users\\gopij'